In [18]:
%load_ext autoreload
%autoreload 2

import os, copy
from pprint import pprint

import pickle
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.transforms import Affine2D

# New implementation
from detection_eval.detection_eval import DetectionEval
from detection_eval.filter import build_kitti_filters, ClassFilter, RangeFilter, CombinedFilter
from detection_eval.box_list import BoxList

print(os.getcwd())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/matthew/git/cadc_testing/uncertainty_eval


In [19]:
dataset_path = '/home/matthew/git/cadc_testing/WISEOpenLidarPerceptron/data/kitti'

# Clustering
# logdir = '.'
logdir = '/home/matthew/git/cadc_testing/pcdet_output/output_pkls'
preds_path = os.path.join(logdir, 'ensemble_a.pkl')
# preds_path = os.path.join(logdir, 'openpcdet_80_result.pkl')
# preds_path = os.path.join(logdir, 'mimo_a_training.pkl')

# gts_path = os.path.join(logdir, 'gts.pkl')
gts_path = os.path.join(logdir, 'kitti_infos_val.pkl')
# gts_path = os.path.join(logdir, 'kitti_infos_train.pkl')


In [20]:
def load_dicts():
    # Load gt and prediction data dict
    with open(gts_path, 'rb') as f:
        gt_dicts = pickle.load(f)
    with open(preds_path, 'rb') as f:
        pred_dicts = pickle.load(f)
    return gt_dicts, pred_dicts

In [21]:
# Load example GT and prediction
idx = np.random.randint(3700)
# idx = 2322, 291
idx = 0
print(idx)
gt_dicts, pred_dicts = load_dicts()

print('Example GT dict:')
print(gt_dicts[idx].keys())
print('Example pred dict:')
print(pred_dicts[idx][0].keys())

0
Example GT dict:
dict_keys(['point_cloud', 'image', 'calib', 'annos'])
Example pred dict:
dict_keys(['name', 'truncated', 'occluded', 'alpha', 'bbox', 'dimensions', 'location', 'rotation_y', 'score', 'score_all', 'boxes_lidar', 'pred_labels', 'target_labels', 'pred_vars', 'pred_head_ids', 'frame_id'])


In [22]:
# Cluster pred_dicts
from cluster import cluster_preds

MIN_CLUSTER_SIZE = 3

print("Before clustering")
print("Number of frames", len(pred_dicts))
print("Number of pred dicts per frame", len(pred_dicts[0]))

pred_dicts = cluster_preds(pred_dicts, MIN_CLUSTER_SIZE)

print("After clustering")
print("Number of frames", len(pred_dicts))
print("Is the output per frame now a single dict?", isinstance(pred_dicts[0], dict))

Before clustering
Number of frames 3769
Number of pred dicts per frame 4
EMPTY FRAME DETECTED
EMPTY FRAME DETECTED
EMPTY FRAME DETECTED
After clustering
Number of frames 3769
Is the output per frame now a single dict? True


In [23]:
print(gt_dicts[2])
print(pred_dicts[2])

{'point_cloud': {'num_features': 4, 'lidar_idx': '000004'}, 'image': {'image_idx': '000004', 'image_shape': array([ 375, 1242], dtype=int32)}, 'calib': {'P2': array([[721.53771973,   0.        , 609.55932617,  44.85728073],
       [  0.        , 721.53771973, 172.85400391,   0.21637911],
       [  0.        ,   0.        ,   1.        ,   0.00274588],
       [  0.        ,   0.        ,   0.        ,   1.        ]]), 'R0_rect': array([[ 0.9999239 ,  0.00983776, -0.00744505,  0.        ],
       [-0.0098698 ,  0.9999421 , -0.00427846,  0.        ],
       [ 0.00740253,  0.00435161,  0.9999631 ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]],
      dtype=float32), 'Tr_velo_to_cam': array([[ 0.00753374, -0.99997139, -0.0006166 , -0.00406977],
       [ 0.01480249,  0.00072807, -0.99989021, -0.07631618],
       [ 0.99986207,  0.00752379,  0.01480755, -0.27178061],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])}, 'annos': {'name': array(['Car', 'Car

In [ ]:
for i in range(len(pred_dicts)):
    num_objs = len(pred_dicts[i]['name'])
    frame_id = pred_dicts[i]['frame_id']
    print(frame_id)
    print(gt_dicts[i]['image']['image_idx'])
#     if num_objs == 0:
#         print(num_objs)

In [29]:
print(pred_dicts[0]['frame_id'])

000001


In [24]:
def pcdet_get_labels(data_dict):
    if isinstance(data_dict, list):
        data_dict = data_dict[0]
    for label_key in ['labels', 'gt_labels', 'pred_labels']:
        if label_key in data_dict:
            return data_dict[label_key]
    if 'annos' in data_dict:
        data_dict = data_dict['annos']
    if 'name' in data_dict:
        classes = dict(
            Car=1, Pedestrian=2, Cyclist=3,
            Truck=-1, Misc=-1, Van=-1, Tram=-1, Person_sitting=-1,
            DontCare=-1
        )
        labels = []
        for name in data_dict['name']:
            if name == 'DontCare':
                continue
            labels.append(classes[name])
        return labels
    raise ValueError()

def pcdet_get_scores(data_dict):
    if isinstance(data_dict, list):
        data_dict = data_dict[0]
    if 'score' in data_dict:
        return data_dict['score']

def pcdet_get_boxes(data_dict, gt_mode=True):
    if isinstance(data_dict, list):
        data_dict = data_dict[0]
    if 'annos' in data_dict:
        data_dict = data_dict['annos']
    for box_key in ['gt_boxes', 'gt_boxes_lidar', 'boxes_lidar', 'box3d_lidar']:
        if box_key in data_dict:
            if gt_mode:
                return data_dict[box_key]
            tmp = copy.deepcopy(data_dict[box_key])
            for i in range(len(tmp)):
                tmp[i][2] += tmp[i][5]/2.0
            return tmp
    return ValueError()

def gt_processor(data_dict):
    return (
        pcdet_get_labels(data_dict),
        pcdet_get_boxes(data_dict)
    )

def pred_processor(data_dict):
    return (
        pcdet_get_labels(data_dict),
        pcdet_get_scores(data_dict),
        pcdet_get_boxes(data_dict, gt_mode=False)
    )

def get_dist(data_dict):
    boxes = get_boxes(data_dict)
    coords = boxes[:,:2]
    dist = np.linalg.norm(coords, axis=1)
    return dist

# Settings for both tests

In [27]:
################################################################################
# 
# Example: evaluating one sample
#
################################################################################

# Threshold (list or dict) maps a label to a matching threshold
# thresholds[label] = threshold
thresholds = {
    -1: 1,
    0: 1,    # Just to not output error
    1: 0.7,    # Car
    2: 0.5,    # Pedestrian
    3: 0.5     # Cyclist
}

# Examples for some common filters
# ClassFilter: filter by class, needs to specify class label
# RangeFilter: filter numerical value by a range
# CombinedFilter: combine multiple filters together with an `AND` clause

car_filter = ClassFilter(name='Car', label=1,
                         gt_processor=gt_processor, pred_processor=pred_processor)

from detection_eval.filter import build_kitti_filters
# kitti_filter_list = build_kitti_filters(logdir + '/kitti_infos_val.pkl')
kitti_filter_list = build_kitti_filters(logdir + '/kitti_infos_train.pkl')
filter_list = [
    kitti_filter_list[1]#, # Car moderate
#     kitti_filter_list[4], # Ped moderate
#     kitti_filter_list[7], # Cyc moderate
]

car_filter = kitti_filter_list[1]

def attach_data(sample_idx, gt_dict, pred_dict, gt_list, pred_list):
    for i in range(len(gt_list)):
        gt_list.data[i] = dict(gt_boxes=pcdet_get_boxes(gt_dict)[i])
    for i in range(len(pred_list)):
        pred_list.data[i] = dict(
#             score_all=pred_dict['score_all'][i],
            boxes_lidar=pred_dict['boxes_lidar'][i]#,
#             pred_vars=pred_dict['pred_vars'][i]
        )


# Evaluate individual frames to check for weird matches

In [30]:
tp_count = 0
for idx in range(3769):
    # Call `evaluate_one_sample` to get the results
    gt_list, pred_list = DetectionEval.evaluate_all_samples(
        [gt_dicts[idx]],
        [pred_dicts[idx]],
        thresholds,
        criterion='iou_3d',
        epsilon=0.1,
        filta=car_filter,
        gt_processor=gt_processor,
        pred_processor=pred_processor,
        callback=attach_data
    )

    # A prediction box is either a TP or FP
    # TP is valid, localized and classified
    tp = pred_list.valid & pred_list.localized & pred_list.classified
    # FP is valid and either not localized or not classified correctly
    fp = pred_list.valid & ~(pred_list.localized & pred_list.classified)
    tp_list = pred_list[tp]
    
    tp_count += len(tp_list)
    print('tp_count', tp_count)

    count = 0
    bad_idx = []
    for j in range(len(tp_list)):
        obj = tp_list[j]
        pred_x = obj.data['boxes_lidar'][0]
        gt_x = gt_list[int(obj.matched_idx)].data['gt_boxes'][0]
        if np.abs(pred_x - gt_x) > 10.0:
            count += 1
            print('find object at index', j)
            bad_idx.append(j)
            print(obj)
            print(obj.data['boxes_lidar'])
            print(obj.matched_idx)
            print(gt_list[int(obj.matched_idx)].data['gt_boxes'])
    if count > 0:
        print('total count', count)
        print('bad_idx', bad_idx)

print('tp count', tp_count)

KeyError: '000001'

# Evaluate all frames to check for weird matches

In [31]:
np.set_printoptions(suppress=True)

gt_list, pred_list = DetectionEval.evaluate_all_samples(
    gt_dicts,
    pred_dicts,
    thresholds,
    criterion='iou_3d',
    epsilon=0.1,
    filta=car_filter,
    gt_processor=gt_processor,
    pred_processor=pred_processor,
    callback=attach_data
)

# A prediction box is either a TP or FP
# TP is valid, localized and classified
tp = pred_list.valid & pred_list.localized & pred_list.classified
# FP is valid and either not localized or not classified correctly
fp = pred_list.valid & ~(pred_list.localized & pred_list.classified)
tp_list = pred_list[tp]

print(len(tp_list))

count = 0
bad_idx = []
for j in range(len(tp_list)):
    obj = tp_list[j]
    pred_x = obj.data['boxes_lidar'][0]
    gt_x = gt_list[int(obj.matched_idx)].data['gt_boxes'][0]
    if np.abs(pred_x - gt_x) > 10.0:
        count += 1
        print('find object at index', j)
        bad_idx.append(j)
        print('obj', obj)
        print('obj.valid', obj.valid)
        print('obj.data boxes_lidar', obj.data['boxes_lidar'])
        print('matched idx', obj.matched_idx)
        print('gt', gt_list[int(obj.matched_idx)])
        print('gt box', gt_list[int(obj.matched_idx)].data['gt_boxes'])
if count > 0:
    print('total count', count)
    print('bad_idx', bad_idx)

KeyError: '000001'

In [33]:
# print(gt_dicts[1994:1997])
# print(pred_dicts[1994:1997])
gt_list, pred_list = DetectionEval.evaluate_all_samples(
    gt_dicts[1994:1997],
    pred_dicts[1994:1997],
    thresholds,
    criterion='iou_3d',
    epsilon=0.1,
    filta=car_filter,
    gt_processor=gt_processor,
    pred_processor=pred_processor,
    callback=attach_data
)

# gt_list, pred_list = DetectionEval.evaluate_all_samples(
#     gt_dicts,
#     pred_dicts,
#     thresholds,
#     criterion='iou_3d',
#     epsilon=0.1,
#     filta=car_filter,
#     gt_processor=gt_processor,
#     pred_processor=pred_processor,
#     callback=attach_data
# )

# A prediction box is either a TP or FP
# TP is valid, localized and classified
tp = pred_list.valid & pred_list.localized & pred_list.classified
# FP is valid and either not localized or not classified correctly
fp = pred_list.valid & ~(pred_list.localized & pred_list.classified)
tp_list = pred_list[tp]

print(len(tp_list))

count = 0
bad_idx = []
for j in range(len(tp_list)):
    obj = tp_list[j]
    pred_x = obj.data['boxes_lidar'][0]
    gt_x = gt_list[int(obj.matched_idx)].data['gt_boxes'][0]
    if np.abs(pred_x - gt_x) > 10.0:
        count += 1
        print('find object at index', j)
        bad_idx.append(j)
        print('obj', obj)
        print('obj.valid', obj.valid)
        print('obj.data boxes_lidar', obj.data['boxes_lidar'])
        print('matched idx', obj.matched_idx)
        print('gt', gt_list[int(obj.matched_idx)])
        print('gt box', gt_list[int(obj.matched_idx)].data['gt_boxes'])
if count > 0:
    print('total count', count)
    print('bad_idx', bad_idx)

KeyError: '004040'